In [22]:
import os
import json
import requests
import yfinance as yf
import pandas as pd

import datetime
from datetime import date, timedelta

from sklearn.preprocessing import MinMaxScaler

In [23]:
start_date = date(2023, 12, 1)
end_date = date(2024, 3, 31)

input_yf = "BBRI.JK"
data = yf.download(input_yf, start=start_date, end=end_date)

# Make date as a column
data.insert(0, "Date", data.index, True)
data.reset_index(drop=True, inplace=True)

# Print data information
print(f"Date from: {start_date} to: {end_date}")
print(data)

columns = "Close"
data = data[["Date", columns]]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[columns].values.reshape(-1, 1))

# Split data into train and test sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train_data, test_data = scaled_data[:train_size, :], scaled_data[train_size:, :]


c:\Users\kevin\miniconda3\envs\tf\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

Date from: 2023-12-01 to: 2024-03-31
         Date    Open    High     Low   Close    Adj Close     Volume
0  2023-12-01  5275.0  5375.0  5275.0  5350.0  5077.938965   99499500
1  2023-12-04  5450.0  5575.0  5450.0  5500.0  5220.311035  235595700
2  2023-12-05  5425.0  5500.0  5425.0  5450.0  5172.854004  114689900
3  2023-12-06  5425.0  5450.0  5400.0  5425.0  5149.125000  119716700
4  2023-12-07  5400.0  5475.0  5400.0  5475.0  5196.582520  102126600
..        ...     ...     ...     ...     ...          ...        ...
72 2024-03-22  6050.0  6125.0  6050.0  6125.0  6125.000000   46529200
73 2024-03-25  6250.0  6250.0  6125.0  6250.0  6250.000000  115729600
74 2024-03-26  6300.0  6300.0  6200.0  6300.0  6300.000000  120748400
75 2024-03-27  6300.0  6325.0  6225.0  6250.0  6250.000000   73308900
76 2024-03-28  6100.0  6200.0  6025.0  6050.0  6050.000000  288926900

[77 rows x 7 columns]


In [24]:
# Define the API endpoint
api_url = 'http://localhost:8501/v1/models/lstm_stock:predict'
 
# Set the request headers
headers = {"content-type": "application/json"}
 

In [25]:
signature_name = "serving_default"
# Consider the first 10 data_test images 
instances = test_data.reshape(-1, 1, 1).tolist()
 
# Create a dictionary
data_dict = {
    "signature_name": signature_name,
    "instances": instances
}

In [30]:
# dir(instances)

In [31]:
data = json.dumps(data_dict)
data

'{"signature_name": "serving_default", "instances": [[[0.75]], [[0.8181818181818183]], [[0.8409090909090908]], [[0.9545454545454541]], [[1.0]], [[0.7727272727272725]], [[0.6136363636363633]], [[0.6363636363636358]], [[0.6363636363636358]], [[0.7272727272727275]], [[0.7272727272727275]], [[0.75]], [[0.8636363636363633]], [[0.9090909090909092]], [[0.8636363636363633]], [[0.6818181818181817]]]}'

In [32]:
# Send a POST request to the API with the JSON data
response = requests.post(api_url, data=data, headers=headers)

In [33]:
response

<Response [200]>

In [34]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response and extract predictions
    response_data = json.loads(response.text)
    predictions = response_data['predictions']
else:
    print(f"Failed to make a request. Status code: {response.status_code}")